In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix


In [3]:
bg_df = pd.read_csv('BIOGRID-ORGANISM-Saccharomyces_cerevisiae.csv')
bg_genes_df = pd.read_csv('BIOGRID-ORGANISM-Saccharomyces_cerevisiae.csv' , usecols = ['Official Symbol Interactor A','Official Symbol Interactor B'])
ess_df = pd.read_csv('scerevisiae_essentiality.csv')

C:\Users\Burcu\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,2,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
 #  bg_df.head()
bg_genes_df.head()

,Official Symbol Interactor A,Official Symbol Interactor B
0,CDC73,CTR9
1,CTR9,CDC73
2,CDC73,LEO1
3,LEO1,CDC73
4,CDC73,PAF1


In [5]:
#bg_df.drop(['Synonyms Interactor A', 'Synonyms Interactor B', 'Experimental System',
#         'Experimental System Type', 'Author', 'Publication Source', 'Throughput',
#         'Modification', 'Qualifications', 'Tags', 'Source Database', 'Ontology Term Names',
#         'Ontology Term Categories', 'Ontology Term Qualifier IDs', 'Ontology Term Qualifier Names',
#         'Ontology Term Types', ], 
#          axis=1, 
#          inplace=True)
ess_df.head()

,Official_Symbol_Interactor,Score,Essentiality
0,MED6,0.36,C
1,SMC2,0.36,C
2,MED8,0.00,NE
3,MED4,0.00,NE
4,MED11,1.00,E


In [6]:
# bg_df.head()
# ess_df.loc[ess_df['Essentiality'] == -1, 'Essentiality'] = 'C'
# ess_df.loc[ess_df['Essentiality'] == 'NE', 'Essentiality'] = 'NE'
# ess_df.loc[ess_df['Essentiality'] == 'E', 'Essentiality'] = 'E'
ess_df.dtypes

Official_Symbol_Interactor     object
Score                         float64
Essentiality                   object
dtype: object

In [7]:
ess_df.head()

,Official_Symbol_Interactor,Score,Essentiality
0,MED6,0.36,C
1,SMC2,0.36,C
2,MED8,0.00,NE
3,MED4,0.00,NE
4,MED11,1.00,E


In [8]:
bg_genes_df.dtypes
bg_genes_df.head()

,Official Symbol Interactor A,Official Symbol Interactor B
0,CDC73,CTR9
1,CTR9,CDC73
2,CDC73,LEO1
3,LEO1,CDC73
4,CDC73,PAF1


In [9]:
# iseq = ess_df['Official_Symbol_Interactor'] == csv2['Official Symbol Interactor A']
# csv3 = pd.DataFrame(csv2[iseq])
# ess_col = ess_df['Official_Symbol_Interactor']
# bg_col_a = bg_genes_df['Official Symbol Interactor A']
# bg_col_b = bg_genes_df['Official Symbol Interactor B']
# bg_ess_df = bg_genes_df
# bg_ess_df["ColumnsMatch"] = np.where(ess_col == bg_col_a, 'True', 'False')

#filter rows based on that column and remove the extra column
# bg_ess_df = bg_ess_df[bg_ess_df['ColumnsMatch'] == 'True'].drop('ColumnsMatch', 1)
            
bg_ess_df = bg_genes_df
bg_ess_df['Essentiality'] = list(range(len(bg_ess_df.index)))  

In [10]:
bg_ess_df.head() # will be adding essentiality here according to the matching genes 

,Official Symbol Interactor A,Official Symbol Interactor B,Essentiality
0,CDC73,CTR9,0
1,CTR9,CDC73,1
2,CDC73,LEO1,2
3,LEO1,CDC73,3
4,CDC73,PAF1,4


In [11]:
# ess_df['Official_Symbol_Interactor'].isin(bg_ess_df['Official Symbol Interactor A'])
bg_ess_df.columns = bg_ess_df.columns.str.replace(' ', '_')
bg_ess_df.head()

,Official_Symbol_Interactor_A,Official_Symbol_Interactor_B,Essentiality
0,CDC73,CTR9,0
1,CTR9,CDC73,1
2,CDC73,LEO1,2
3,LEO1,CDC73,3
4,CDC73,PAF1,4


In [12]:
print (bg_ess_df[bg_ess_df.Official_Symbol_Interactor_A.isin(ess_df.Official_Symbol_Interactor)])

       Official_Symbol_Interactor_A Official_Symbol_Interactor_B  Essentiality
60                             DSN1                         AME1            60
61                             DSN1                         CHL4            61
62                             DSN1                        CTF19            62
63                             DSN1                         CTF3            63
64                             DSN1                        MCM22            64
...                             ...                          ...           ...
783460                         MEC1                         SAE2        783460
783461                         MEC1                        RAD50        783461
783463                         MEC1                        MRE11        783463
783468                         BRN1                         SMC4        783468
783469                         SMC4                         BRN1        783469

[79406 rows x 3 columns]


In [13]:
print (bg_ess_df[bg_ess_df.Official_Symbol_Interactor_B.isin(ess_df.Official_Symbol_Interactor)])

       Official_Symbol_Interactor_A Official_Symbol_Interactor_B  Essentiality
42                             CWC2                        PRP45            42
43                             CWC2                        PRP46            43
49                             CWC2                       SNU114            49
50                             CWC2                         SYF1            50
52                             CWC2                         YJU2            52
...                             ...                          ...           ...
783377                         SSD1                         MPS3        783377
783383                         SSD1                        RPB11        783383
783387                         SSD1                         ATG1        783387
783468                         BRN1                         SMC4        783468
783469                         SMC4                         BRN1        783469

[57620 rows x 3 columns]


In [14]:
ess_df=ess_df.rename(columns = {'Official_Symbol_Interactor':'Official_Symbol_Interactor_A'}) # change the column name of ess_df to be able to use merge
print (pd.merge(bg_ess_df,ess_df, on='Official_Symbol_Interactor_A'))

       Official_Symbol_Interactor_A Official_Symbol_Interactor_B  \
0                              DSN1                         AME1   
1                              DSN1                         CHL4   
2                              DSN1                        CTF19   
3                              DSN1                         CTF3   
4                              DSN1                        MCM22   
...                             ...                          ...   
129753                         BCD1                       RTT106   
129754                         BCD1                       RTT106   
129755                         BCD1                       RTT106   
129756                         ECM9                         RCE1   
129757                         ECM9                      YNL320W   

        Essentiality_x  Score Essentiality_y  
0                   60    1.0              E  
1                   61    1.0              E  
2                   62    1.0             

In [15]:
print (pd.merge(bg_ess_df,ess_df, on='Official_Symbol_Interactor_A')['Essentiality_y'].unique())

['E' 'C' 'NE']


In [16]:
print (pd.merge(bg_ess_df,ess_df, on='Official_Symbol_Interactor_A')['Score'].unique())

[1.         0.5625     0.         0.25       0.44444444 0.64
 0.16       0.11111111 0.36       0.69444444 0.0625    ]


In [17]:
len(bg_ess_df)

783485

In [18]:
print (pd.merge(bg_ess_df,ess_df, on='Official_Symbol_Interactor_A')['Official_Symbol_Interactor_A'])

0         DSN1
1         DSN1
2         DSN1
3         DSN1
4         DSN1
          ... 
129753    BCD1
129754    BCD1
129755    BCD1
129756    ECM9
129757    ECM9
Name: Official_Symbol_Interactor_A, Length: 129758, dtype: object


In [19]:
# return all rows from df2 having values of df1
# return all values from BIOGRID-ORGANISM having values of scerevisiae_essentiality
# print(bg_ess_df.merge(ess_df,left_on=['Official_Symbol_Interactor_A','Essentiality'],right_on=['Official_Symbol_Interactor_A','Essentiality'],how='left').dropna()[bg_ess_df.columns]
# )
bg_ess_df.to_csv('bg_ess.csv', index = None)

In [20]:
# file_path = 'C:\Users\Burcu\Desktop\nt\xgboost_ppi\ppiemb.txt'
# dataframe1 = pd.read_csv(file_path, delim_whitespace=True)

# dataframe1.to_csv('ppiemb.csv', index = None)

dataframe1 = pd.read_csv("ppiemb.txt", delim_whitespace=True) 
dataframe1.to_csv('ppiemb.csv', index = None)
ppiemb_df = pd.read_csv('ppiemb.csv')


In [21]:
ppiemb_df.dtypes

33604            int64
-0.510713      float64
-0.0346978     float64
0.226693       float64
-0.104328      float64
                ...   
-0.177385      float64
0.000920155    float64
0.0914044      float64
0.0578524      float64
0.0370801      float64
Length: 129, dtype: object